In [11]:
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlopen,Request
from urllib.parse import urlparse
import re
import json
import csv
from urllib.error import HTTPError,URLError
from time import sleep
import pandas as pd

def getTextFromHTML(ans, names):
	if ans == None:
		return None
	else:
		ans = ans.find(names[0],{names[1]:names[2]})
		if ans == None:
			return None
	return ans.get_text()

key_tags_dict = {
	"author":["a","class","a-size-base a-link-normal author"],
	"title":["a","class","a-size-base a-link-normal review-title a-color-base a-text-bold"],
	"rating":["i","",""],
	"size":["a","class","a-size-mini a-link-normal a-color-secondary"],
	"text":["span","class","a-size-base review-text"],
	"vote":["span","class","review-votes"],
	"date":["span","class","a-size-base a-color-secondary review-date"],
}


col_list = [
	"author",
	"date",
	"rating",
	"size",
	"text",
	"title",
	"vote"
]

df_in = pd.read_csv("amazon_asin_list.csv", header=None)
asin_list = df_in[0].tolist()

for ASIN in asin_list:

	print(ASIN)
	try:
		html = urlopen("https://www.amazon.co.jp/product-reviews/"+ASIN)

		review_list = []

		bsObj = BeautifulSoup(html,"lxml")
		pageNumbers = bsObj.findAll("li",{"class":"page-button"})
		print(pageNumbers)
		numOfPages = 0
		for pageNumber in pageNumbers:
			numOfPages=max(numOfPages,int(pageNumber.get_text()))



		# for num in range(0,numOfPages):#TODO:後で直す
		for num in range(0,1):
			success = True
			for _ in range(0,3):
				sleep(10)
				try:
					url = "https://www.amazon.co.jp/product-reviews/"+ASIN+"/pageNumber="+str(1+num)
					html = urlopen(url)
					success = True
					break
				except (HTTPError,URLError) as e:
					print("error : {0}".format(e))
					print("reconnecting..")
					sleep(10)
					success = False
			if not success:
				print("Failed to retrieve "+url+".retrieving next page")
				continue
			print("retrieving {0}/{1}...".format(1+num, numOfPages))
			bsObj = BeautifulSoup(html,"lxml")
			for review in bsObj.findAll("div",{"class":"a-section review"}):
				text_list=[]
				for key in sorted(key_tags_dict.keys()):
					text = getTextFromHTML(review, key_tags_dict[key])
					text_list.append(text)


				if text_list[2] and "5つ星のうち" in text_list[2]:
					text_list[2] = text_list[2].replace("5つ星のうち","")

				if text_list[6] and "人のお客様がこれが役に立ったと考えています" in text_list[6]:
					text_list[6] = text_list[6].replace("人のお客様がこれが役に立ったと考えています.","").replace("\n","")

				print(text_list)
				review_list.append(text_list)

		df_review = pd.DataFrame(review_list,columns = col_list)
		df_review.to_csv(ASIN+"_amazon.csv")
	except HTTPError as e:
		print("ASIN is wrong. : {0}".format(e))

	print (ASIN + " done.")


B0114N2Q94
[<li class="a-selected page-button" data-reftag="cm_cr_arp_d_paging_btm"><a href="/アリエール-イオンパワージェル-サイエンスプラス-超特大サイズ-1-35kg/product-reviews/B0114N2Q94">1</a></li>, <li class="page-button" data-reftag="cm_cr_arp_d_paging_btm"><a href="/アリエール-イオンパワージェル-サイエンスプラス-超特大サイズ-1-35kg/product-reviews/B0114N2Q94?pageNumber=2">2</a></li>, <li class="page-button" data-reftag="cm_cr_arp_d_paging_btm"><a href="/アリエール-イオンパワージェル-サイエンスプラス-超特大サイズ-1-35kg/product-reviews/B0114N2Q94?pageNumber=3">3</a></li>, <li class="page-button" data-reftag="cm_cr_arp_d_paging_btm"><a href="/アリエール-イオンパワージェル-サイエンスプラス-超特大サイズ-1-35kg/product-reviews/B0114N2Q94?pageNumber=4">4</a></li>, <li class="page-button" data-reftag="cm_cr_arp_d_paging_btm"><a href="/アリエール-イオンパワージェル-サイエンスプラス-超特大サイズ-1-35kg/product-reviews/B0114N2Q94?pageNumber=34">34</a></li>]
retrieving 1/34...
['Amazon カスタマー', '2016年2月7日', '1.0', 'サイズ: 詰替 1350g ×6個', '気をつけてください！詰め替え、まとめ買いで安いかと思いきや、1Ｌボトルの方が安いです。アリエールは詰替えも量が違うものがいくつかありますので、購入の際は1Ｌあたりの価格を比較しないと損します

In [14]:
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlopen,Request
from urllib.parse import urlparse
import re
import json
import csv
from urllib.error import HTTPError,URLError
from time import sleep
import pandas as pd

def getTextFromHTML(ans, names):
	if ans == None:
		return None
	else:
		ans = ans.find(names[0],{names[1]:names[2]})
		if ans == None:
			return None
	return ans.get_text()

key_tags_dict = {
	"author":["a","class","a-size-base a-link-normal author"],
	"title":["h3","",""],
	"rating":["dd","class","numeric"],
	"text":["span","class","a-size-base review-text"],
	"date":["span","class","a-size-base a-color-secondary review-date"],
}


col_list = [
	"author",
	"date",
	"rating",
	"text",
	"title",
]

df_in = pd.read_csv("lohaco_id_list.csv", header=None)
asin_list = df_in[0].tolist()

for lohaco_id in asin_list:

	print(lohaco_id)
	try:
		html = urlopen("http://lohaco.jp/product/"+str(lohaco_id)+"/review/")

		review_list = []

		bsObj = BeautifulSoup(html,"lxml")
		pageNumbers = bsObj.findAll("li",{"class":"page-button"})
		print(pageNumbers)
		numOfPages = 0
		for pageNumber in pageNumbers:
			numOfPages=max(numOfPages,int(pageNumber.get_text()))



		# for num in range(0,numOfPages):#TODO:後で直す
		for num in range(0,1):
			success = True
			for _ in range(0,3):
				sleep(10)
				try:
					url = "http://lohaco.jp/product/"+str(lohaco_id)+"/review/?page="+str(1+num)
					html = urlopen(url)
					success = True
					break
				except (HTTPError,URLError) as e:
					print("error : {0}".format(e))
					print("reconnecting..")
					sleep(10)
					success = False
			if not success:
				print("Failed to retrieve "+url+".retrieving next page")
				continue
			print("retrieving {0}/{1}...".format(1+num, numOfPages))
			bsObj = BeautifulSoup(html,"lxml")
			for review in bsObj.findAll("div",{"class":"cfx"}):
				text_list=[]
				for key in sorted(key_tags_dict.keys()):
					text = getTextFromHTML(review, key_tags_dict[key])
					text_list.append(text)


				if text_list[2] and "5つ星のうち" in text_list[2]:
					text_list[2] = text_list[2].replace("5つ星のうち","")

				if text_list[6] and "人のお客様がこれが役に立ったと考えています" in text_list[6]:
					text_list[6] = text_list[6].replace("人のお客様がこれが役に立ったと考えています.","").replace("\n","")

				print(text_list)
				review_list.append(text_list)

		df_review = pd.DataFrame(review_list,columns = col_list)
		df_review.to_csv(lohaco_id+"_amazon.csv")
	except HTTPError as e:
		print("lohaco_id is wrong. : {0}".format(e))

	print (lohaco_id + " done.")


1856283
[]
retrieving 1/0...


IndexError: list index out of range